In [1]:
# changing core directory
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
os.chdir('..')

%load_ext autoreload
%autoreload

In [2]:
from collections import defaultdict

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import ndcg_score
import lightgbm as lgb
from tqdm import tqdm

from src.data import get_sequences, load_data, download_movielens1m, ValSASRecDataset, TrainSASRecDataset
from src.utils import fix_seed
from src.model import SASRec


%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [4]:
seed = 0
max_len = 200
batch_size = 128
embedding_size = 64

n_ext_users = 10

In [5]:
download_movielens1m("data/movielens_1m")
ratings = load_data("data/movielens_1m")

ratings = ratings[ratings["rating"] > 3.5]

# Подготовка данных
num_users = ratings['user_id'].nunique()
num_items = ratings['item_id'].nunique()

user2id = {val:i for i, val in enumerate(ratings['user_id'].unique())}
item2id = {val:i+1 for i, val in enumerate(ratings['item_id'].unique())}

ratings['user_id'] = ratings['user_id'].map(user2id)
ratings['item_id'] = ratings['item_id'].map(item2id)

(train_sequences, val_sequences), (train_times, val_times), (train_users, val_users) = get_sequences(ratings)

Dataset already exists.


In [6]:
train_dataset = TrainSASRecDataset(train_sequences, train_times, max_len)
val_dataset = ValSASRecDataset(val_sequences, val_times, max_len)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [8]:
from torchsummary import summary
model = SASRec(num_items, embedding_size=embedding_size, max_len=max_len)

model.freeze()
# summary(model, (28))

Все слои сети заморожены.


In [11]:
one_batch = next(iter(train_loader))

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

(input_seqs, train_pad_mask, input_times), (targets, label_pad_mask) = one_batch 
targets = targets - 1
optimizer.zero_grad()
outputs = model(input_seqs, train_pad_mask, input_times)

outputs, targets, label_pad_mask = outputs.reshape(-1, num_items), targets.reshape(-1), label_pad_mask.reshape(-1)

targets = targets.masked_fill(mask=(~label_pad_mask), value=-100)
# print(outputs.shape, targets.shape, padding_mask.shape)
# outputs, targets = outputs[padding_mask], targets[padding_mask]
# print(outputs.shape, targets.shape)
loss = criterion(outputs, targets)
loss.backward()
optimizer.step()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn